## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [ ]:
for my_int = 1:96

    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min   = jvalue(vars["f_min"])
    
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])
    
    U_min_def = jvalue(vars["U_min_def"])
    
    psi_min_i = jvalue(vars["psi_min_i"])
    psi_min_d = jvalue(vars["psi_min_d"])
    psi_min_p = jvalue(vars["psi_min_p"])
    psi_min_b = jvalue(vars["psi_min_b"])
    psi_min_c = jvalue(vars["psi_min_c"])    
    
    psi_loss_i = jvalue(vars["psi_loss_i"])
    psi_loss_d = jvalue(vars["psi_loss_d"])
    psi_loss_p = jvalue(vars["psi_loss_p"])
    psi_loss_b = jvalue(vars["psi_loss_b"])
    psi_loss_c = jvalue(vars["psi_loss_c"])
    
    marge_dies = jvalue(vars["marge_dies"])
    
    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Minimum frequency limitation
        # X1 - Minimum voltage relaxed limitation
        # X2 - Minimum voltage firm    limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        # B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0
        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s]*(1-psi_loss_p) + sum((P_dies[t,p,s,d] - psi_loss_d*1e3*ON_dies[t,d]) for d=ds) + P_bat_set[t,p]*(1-psi_loss_b) - L_C[t,p,s]*(1+psi_loss_c) - psi_loss_i*1e3) == 0
                
        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]
        
        # X1[t=ts,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s] + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s] >= U_min_def
        X1[t=m1s,p=ps,s=ss], psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*P_PV[t,p,s]                      + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*L_C[t,p,s]                      >= U_min_def
        X2[t=m2s,p=ps],      psi_min_i + sum((psi_min_d*ON_dies[t,d]) for d=ds) + psi_min_p*1e-3*sum(P_PV[t,p,s] for s=ss)/N_scen + psi_min_b*1e-3*P_bat_set[t,p] + psi_min_c*1e-3*sum(L_C[t,p,s] for s=ss)/N_scen >= U_min_def
        
    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
        status    = string(status),
        interval  = interval,
        P_bat_set = getvalue(P_bat_set[:,:]), 
        SOC_bat   = getvalue(SOC_bat[:,:]),
        P_PV_set  = getvalue(P_PV_set[:,:]),
        P_PV      = getvalue(P_PV[:,:,:]),
        ON_dies   = getvalue(ON_dies[:,:]),
        P_dies    = getvalue(P_dies[:,:,:,:]),
        P_bat_cha = getvalue(P_bat_cha[:,:]),
        P_bat_dis = getvalue(P_bat_dis[:,:]),
        X_bat     = getvalue(X_bat[:,:]))
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/"*dec(N_EMS))
    
    if string(status)=="Infeasible"
        println("Interval "*dec(my_int)*" is Infeasible !!!")
        break
    end
    
end

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-05, 1e+04]
Presolve removed 9678 rows and 5143 columns
Presolve time: 1.83s
Presolved: 189042 rows, 98249 columns, 673777 nonzeros
Variable types: 95667 continuous, 2582 integer (2582 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8106404e+07   8.759531e+06   0.000000e+00      6s
   40326    1.7947286e+07   8.458814e+05   0.000000e+00     10s
   68134    1.7905532e+07   5.846839e+05   0.000000e+00     15s
   93066    1.7811402e+07   3.583135e+05   0.000000e+00     20s
  117463    1.7565498e+07   2.101086e+05   0.000000e+00     25s
  141025    1.6166518e+07   9.780148e+05   0.000000e+00     30s
  160999  

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [2e-05, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 131.738625690

Presolve removed 9652 rows and 5196 columns
Presolve time: 1.66s
Presolved: 189068 rows, 98196 columns, 674967 nonzeros
Variable types: 95628 continuous, 2568 integer (2568 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   23120    1.7910711e+07   1.299690e+06   0.000000e+00      5s
   56660    1.7887256e+07   7.089321e+05   0.000000e+00     10s
   84590    1.7836785e+07   4.667583e+05   0.000000e+00     15s
  112444    1.7658478e+07   2.192181e+05   0.000000e+00     20s
  136223    1.7327119e+07   7.

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [1e-05, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 693.363906201

Presolve removed 9613 rows and 5157 columns
Presolve time: 1.81s
Presolved: 189107 rows, 98235 columns, 675102 nonzeros
Variable types: 95658 continuous, 2577 integer (2577 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   19161    1.7795230e+07   2.287033e+06   0.000000e+00      5s
   54879    1.7755974e+07   6.615659e+05   0.000000e+00     10s
   82241    1.7706927e+07   5.041291e+05   0.000000e+00     15s
  109555    1.7537507e+07   3.524940e+05   0.000000e+00     20s
  133932    1.7231790e+07   7.

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [4e-05, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 120.112772201

Presolve removed 9825 rows and 5407 columns
Presolve time: 1.83s
Presolved: 188895 rows, 97985 columns, 674909 nonzeros
Variable types: 95466 continuous, 2519 integer (2519 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   21803    1.8074403e+07   1.206477e+06   0.000000e+00      5s
   56630    1.8050391e+07   6.422627e+05   0.000000e+00     10s
   83170    1.7993754e+07   4.409159e+05   0.000000e+00     15s
  111347    1.7776731e+07   2.164802e+05   0.000000e+00     20s
  137276    1.6470656e+07   2.

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [5e-05, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 673.703699693

Presolve removed 9774 rows and 5309 columns
Presolve time: 1.73s
Presolved: 188946 rows, 98083 columns, 674734 nonzeros
Variable types: 95538 continuous, 2545 integer (2545 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   22140    1.7855195e+07   1.306790e+06   0.000000e+00      5s
   56597    1.7824695e+07   6.665410e+05   0.000000e+00     10s
   81963    1.7780693e+07   4.616400e+05   0.000000e+00     15s
  109064    1.7626971e+07   2.754470e+05   0.000000e+00     20s
  134640    1.7283684e+07   7.

...

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [8e-04, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 239.169724244

Presolve removed 9542 rows and 5522 columns
Presolve time: 1.78s
Presolved: 189178 rows, 97870 columns, 679572 nonzeros
Variable types: 95389 continuous, 2481 integer (2481 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8686479e+07   8.338537e+06   0.000000e+00      6s
   48380    1.8489048e+07   7.416912e+05   0.000000e+00     10s
   74764    1.8411820e+07   5.703177e+05   0.000000e+00     15s
   97830    1.8294638e+07   3.361946e+05   0.000000e+00     20s
  118766    1.8081235e+07   1.

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [8e-05, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 342.907582884

Presolve removed 9366 rows and 5411 columns
Presolve time: 1.65s
Presolved: 189354 rows, 97981 columns, 680524 nonzeros
Variable types: 95467 continuous, 2514 integer (2514 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8911785e+07   8.416120e+06   0.000000e+00      6s
   51454    1.8706539e+07   7.027474e+05   0.000000e+00     10s
   79076    1.8620695e+07   5.936461e+05   0.000000e+00     15s
  103872    1.8456866e+07   2.997664e+05   0.000000e+00     20s
  130495    1.8106148e+07   9.

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [1e-04, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 76.944214066

Presolve removed 9455 rows and 5472 columns
Presolve time: 1.69s
Presolved: 189265 rows, 97920 columns, 680469 nonzeros
Variable types: 95404 continuous, 2516 integer (2516 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8816629e+07   8.419252e+06   0.000000e+00      6s
   51733    1.8613704e+07   7.434090e+05   0.000000e+00     10s
   79740    1.8529369e+07   5.046194e+05   0.000000e+00     15s
  104633    1.8370682e+07   2.683353e+05   0.000000e+00     20s
  130289    1.8001138e+07   8.4

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [8e-01, 2e+04]
  RHS range        [1e-04, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 700.160746484

Presolve removed 9500 rows and 5486 columns
Presolve time: 1.64s
Presolved: 189220 rows, 97906 columns, 679646 nonzeros
Variable types: 95398 continuous, 2508 integer (2508 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8126653e+07   8.362072e+06   0.000000e+00      6s
   52319    1.7929001e+07   7.638945e+05   0.000000e+00     10s
   79232    1.7863609e+07   4.673792e+05   0.000000e+00     15s
  106357    1.7662673e+07   2.498930e+05   0.000000e+00     20s
  131060    1.7299028e+07   7.

Academic license - for non-commercial use only
Optimize a model with 198720 rows, 103392 columns and 729359 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [3e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [6e-01, 2e+04]
  RHS range        [3e-05, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 0.483816752

Presolve removed 9202 rows and 5238 columns
Presolve time: 1.65s
Presolved: 189518 rows, 98154 columns, 680972 nonzeros
Variable types: 95590 continuous, 2564 integer (2564 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7930208e+07   8.419268e+06   0.000000e+00      5s
   51896    1.7745578e+07   7.355152e+05   0.000000e+00     10s
   79559    1.7674209e+07   5.107219e+05   0.000000e+00     15s
  105032    1.7517989e+07   2.911087e+05   0.000000e+00     20s
  131531    1.7145787e+07   1.14